<a href="https://colab.research.google.com/github/javajaewook/Jaewook-Lee/blob/master/Chapter_6_SVMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2><b>SMO 알고리즘으로 효율적인 최적화 하기</b></h2>

*  SMO 알고리즘은 α와 b의 집합을 찾는 것
*  적당한 두 개의 α를 선택해서 각각의 α의 값을 조절

<br>

α 집합은 아래의 두 조건을 충족해야 함.

*   α쌍 모두가 그들의 마진 경계 밖에 있어야 함
*   α가 이미 고정되어 있거나 경계를 갖지 않아야 함

<br><br>




아래는 SMO 알고리즘을 위해 사용될 함수들이다.






In [0]:
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

def selectJrand(i,m):
    j=i # j랑 i가 같으면 반복
    while (j==i):
        j = int(random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

loadDataSet()은 파일을 열고 각 줄을 분류 항목 표시와 데이터 행렬로 분석하는 역할을 한다.
<br>

SelectJrand()는 j가 i와 다른 값을 가질 때까지 0에서 m 사이의 실수를 무작위로 선택해서 j에 대입하고, 대입한 j를 반환한다.
<br>

clipAlpha()는 aj(alpha j)가 H와 L사이의 값이 되도록 보정하는 역할을 한다.
<br><br><br>

위 함수의 실험을 위해 아래의 코드를 실행시키고, 결과를 확인해보자. 
<br>

(Colab에서 드라이브의 파일을 사용하기 위해서 아래의 코드도 함께 실행한다)

In [0]:
dataArr, labelArr = loadDataSet(link + 'testSet.txt')
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
link = '/content/gdrive/My Drive/Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


분류 항목 표시가 이전의 0과 1이 아니라, 1과 -1로 되어있는 것을 볼 수 있다. 이는 지지 벡터 머신에서 마진의 최적화를 위해 1과 -1의 분류 표기를 사용하기 때문이다.
<br><br>


이제 SMO 알고리즘을 시도해보자.
<br>



In [0]:
from numpy import *
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn); labelMat = mat(classLabels).transpose()
    b = 0; m,n = shape(dataMatrix)
    alphas = mat(zeros((m,1))) # 데이터가 가진 속성만큼의 alpha가 필요하다
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m): 
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b # 예측 계산
            Ei = fXi - float(labelMat[i]) # 오류 계산
            
            # SMO 조건 확인 
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m) # j를 선택
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b # 예측 계산
                Ej = fXj - float(labelMat[j]) # 오류 계산
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy(); # 두 알파 값을 변경하기 이전에 보존
                if (labelMat[i] != labelMat[j]): # 두 데이터의 관측치가 다르면
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print("L==H"); continue # L과 H가 같으면 더 이상 변경 불가하므로 countinue
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T # alpha[j]를 변경하기 위한 계수 eta 계산
                if eta >= 0: print("eta>=0"); continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta # 새로운 j를 계산
                alphas[j] = clipAlpha(alphas[j],H,L) # j의 값이 범위 내에 있는지 확인
                if (abs(alphas[j] - alphaJold) < 0.00001): print ("j not moving enough"); continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])# j가 변경된만큼 i도 변경
                                                                            # 변경은 j의 반대 방향으로 일어나야 한다
                
                # 두 alpha 값을 최적화한 후, 두 알파에 대한 일정한 간격인 b를 설정
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1 
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1 # 횟수 체크
                print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print ("iteration number: %d" % iter)
    return b,alphas
    

위 함수를 실습해보기 위해서 아래의 코드를 실행시켜보자.

In [0]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

iter: 0 i:0, pairs changed 1
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
L==H
j not moving enough
L==H
iter: 0 i:54, pairs changed 2
iter: 0 i:55, pairs changed 3
L==H
iter: 0 i:76, pairs changed 4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
L==H
L==H
iter: 0 i:22, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
L==H
iter: 0 i:46, pairs changed 2
j not moving enough
j not moving enough
L==H
iter: 0 i:76, pairs changed 3
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
iter: 0 i:22, pairs changed 1
j not moving enough
L==H
L==H
iter: 0 i:29, pairs changed 2
L==H
iter: 0 i:39, pairs changed 3
L==H
j not moving enough
iter: 0 i:55, pairs changed 4
j not moving enough
iter: 0 i:70, pairs changed 5
j not moving enough
L==H
j not moving eno

b와 alpha도 확인해보자.
<br>
 
alphas 내부에는 0이 많으므로 0보다 큰 원소의 수를 확인한다.

In [0]:
b

matrix([[-3.84767954]])

In [0]:
alphas[alphas>0] # 배열 필터링

matrix([[0.10582818, 0.10415197, 0.15067719, 0.11085628, 0.24980107]])

임의로 발생되는 SMO 알고리즘의 특성상 실행시킬 때마다 다른 결과를 얻게 된다.
<br><br>

지지 벡터의 수를 구하기 위해서 아래 코드를 실행시켜보자.

In [0]:
shape(alphas[alphas>0])

(1, 5)

데이터 집합에 있는 지지 벡터가 어떤 값을 가지고, 어떻게 분류되어 있는지도 확인해보자.

In [0]:
for i in range(100):
  if alphas[i] > 0.0 : print(dataArr[i], labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[2.893743, -1.643468] -1.0
[5.286862, -2.358286] 1.0
[6.080573, 0.418886] 1.0


<h2><b>전체 플랫 SMO를 이용해 최적화 속도 올리기</b></h2>
<br>

간략한 형태의 SMO는 작은 데이터 집합에서는 잘 동작하지만, 데이터 집합이 커지면 속도가 떨어진다. 이를 개선하기 위해서 최적화에서 alpha를 선택하는 방법을 변경해 볼 것이다.
<br><br>

첫 번째 alpha를 선택하는 방법은 그대로 두고, 두 번째 alpha를 선택하는 방법을 변경할 것이다. 간략한 형태의 SMO에서는 j를 선택한 후 오류 Ej를 계산했지만, 새 방법에서는 오류 값에 대한 **전역 캐시(global cache)**를 생성할 것이며, **단계 크기** 혹은 **Ei-Ej**를 가장 크게하는 alpha를 선택할 것이다.
<br><br>

이를 도와줄 새로운 도움 함수 3개를 정의할 것이다.
<br><br>

아래의 코드를 확인해보자.

In [0]:
class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) # <유효비트, E값>으로 구성된 캐시
        self.K = mat(zeros((self.m,self.m)))
        
def calcEk(oS, k): # 오류를 계산하는 함수. 자주 쓰일 예정이라 함수로 분리해 놓음
    fXk = float(multiply(oS.alphas,oS.labelMat).T*(oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
def selectJ(i, oS, Ei): # j와 Ej를 계산하는 함수
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  # 유효 비트를 1로   
                           # 최대 deltaE를 만드는 alpha를 선택한다.
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   # deltaE를 최대한 크게하는 Ecache를 찾기 위한 순환문
            if k == i: continue # i는 calcEk()를 거치면 Ei가 된다. Ei-Ei == 0이므로 계산할 필요가 없다.
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   # 계산 가능한 ecache 값이 없는 경우
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k): # 오류를 계산하고 계산한 값을 캐시에 저장. 알파 값을 최적화한 후에 이 값을 사용한다.
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]

<br>

도움 함수가 준비되었으면, inner Loop를 위한 함수 innerL()를 보자.
<br>



In [0]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) # 두 번째 alpha를 선택하기 위해 selectJ()를 사용한다.
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print("L==H"); return 0
        eta = 2.0 * oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - oS.X[j,:]*oS.X[j,:].T
        if eta >= 0: print("eta>=0"); return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) # Ecache를 갱신한다.
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print("j not moving enough");  return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])# j가 움직인만큼 i도 움직여준다.
        updateEk(oS, i) # Ecache를 갱신한다.                                    # 반대 방향으로 움직인다.
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[i,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[j,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

<br>

innerL()을 전체 플랫 SMO 코드에 적용한다.
<br>



In [0]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):    # 전체 플랫 SMO
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   # 모든 값을 검토하는 부분
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print ("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:# 범위에 속하지 않는 값을 검토
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print ("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False # 모든 값의 검토가 끝나면 False로 바꾸어 범위 밖의 값을 볼 수 있도록 한다.
        elif (alphaPairsChanged == 0): entireSet = True  
        print ("iteration number: %d" % iter)
    return oS.b,oS.alphas


smoP()는 smoSimple()과 while 반복문 내부가 다르게 구성된다. 첫 번째 for 반복문은 데이터 집합에 있는 alpha를 검토하지 않는다. 또, 두 번째 alpha를 선택하기 위해 innerL()를 호출하고, 가능하면 최적화를 수행한다. 
<br>

smoP()는 어떤 쌍이든 변화가 발생한다면 1을 반환한다. 
<br><br>

두 번째 for 반복문은 범위에 속하지 않는 alpha를 검토한다.
<br><br>

그 다음 범위가 없는 반복문 부분과 전체를 검토하는 부분을 번갈아 처리하기 위해, entireSet의 값을 바꾸면서 반복 횟수를 출력하고, 마지막으로 상수 b와 alpha를 반환한다.
<br><br>

실습을 위해서 아래의 코드를 실행시켜 보자.


In [0]:
dataArr, labelArr = loadDataSet(link + 'testSet.txt')
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

NameError: ignored

이제 alpha와 b를 구했으니, w를 구할 준비가 다 되었다. w를 구하는 함수는 아래와 같다.
<br>



In [0]:
def calcWs(alphas,dataArr,classLabels):
    X = mat(dataArr); labelMat = mat(classLabels).transpose()
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

가중치 벡터 w를 구하기 위해서 아래의 코드를 실행시켜보자.

In [0]:
ws = calcWs(alphas, dataArr, labelArr)
ws

array([[ 0.81596315],
       [-0.27177788]])

이제 w, b를 전부 알았으니, 어떤 벡터 x가 주어지면, Wx+b의 값을 확인해서 분류를 할 수 있다.

In [0]:
datMat = mat(dataArr)
(datMat[0] * mat(ws)) + b

matrix([[-1.49455537]])

계산한 결과가 1보다 크면 분류 집단 A(1)이고, -1보다 작으면 분류 집단 B(-1)이다.

<br><br>

<h2><b>비선형 분류를 위한 Kernel-SVM</b></h2>

완벽하게 **선형 분류**할 수 없는 데이터 집단을 분류하기 위해서, 기존의 데이터를 더 **고차원**의 공간으로 사상시킨 후, 두 집단을 분류하는 **분류 초평면**을 구하는 방법을 사용한다.

<br><br>
벡터를 고차원으로 사상하기 위해서 커널 함수가 필요한데, 실습에서는 **반지름 성향 함수(Radius Bias function)**을 사용할 것이다.

<br>
아무 함수나 사용할 수 있는 것은 아니고, 선형 변환할 수 있는 함수만 커널 함수로 사용할 수 있다.


*   T(cv) = cT(v) (동차성)
*   T(u+v) = T(u) + T(v) (가산성)

<br>
이 두 가지 조건을 만족하는 함수 T에 의한 변환을 <b>선형 변환</b>이라고 하는데, 이 때, 함수 T를 <b>커널 함수</b>라고 한다.

<br>
커널 함수는 여러가지 특징을 가지는데, 그 중, <b>커널 함수와 커널 함수를 곱해도 커널 함수의 성질을 가진다</b>는 특징을 이용할 것이다.

<br><br>
실습에 사용할 **반지름 성향 함수**에서 **σ**는 **하나의 데이터 샘플이 영향력을 행사하는 거리**이자, **결정 경계의 곡률을 조정**하는 변수이자, 얼마나 빨리 0으로 떨어지는지를 결정하는 변수이다.

<br><br>
관측치 벡터를 커널을 거쳐 고차원으로 사상하는 코드를 보자.

In [0]:
def kernelTrans(X, A, kTup): # 관측치 벡터를 고차원으로 사상하는 함수
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0]=='lin': K = X * A.T   # 선형 커널
    elif kTup[0]=='rbf': # 반지름 성향 함수(가우시안 커널)
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = exp(K/(-1*kTup[1]**2)) # 가우시안 커널 계산
    else: raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):  # 커널에 대한 정보를 담은 인자가 추가되었다.
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m): # 이 부분에서 커널 함수를 거치면 벡터를 고차원으로 사상한 후 내적한 결과를 얻을 수 있다.
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

innerL()과 calcEk()에서 커널 함수를 사용할 수 있도록 코드를 일부 변경하였다.

In [0]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) 
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print ("L==H"); return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] # 커널을 위해 변경한 부분 1. 가우시안 커널을 거친 행렬 K를 사용한다.
        if eta >= 0: print ("eta>=0"); return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) 
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print ("j not moving enough"); return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])
        updateEk(oS, i) 
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j] # 커널을 위해 변경한 부분 2.
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b) # X 대신 K를 사용하도록 바뀌었다.
    Ek = fXk - float(oS.labelMat[k])
    return Ek

이제 관측치를 커널을 사용해 분류면을 구하는 Kernel-SVM 코드를 실행해보자.

In [0]:
def testRbf(k1=1.3):
    dataArr,labelArr = loadDataSet(link + 'testSetRBF.txt')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1)) # C = 200. 여기서 C는 얼마나 많은 데이터 샘플이 다른 분류로 놓이는 것을 허용하는지를 결정한다.
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] # 지지벡터만 사용한다.
    labelSV = labelMat[svInd];
    print ("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1)) # 커널을 사용해서 데이터를 변환한다.
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b # 변환된 데이터를 사용해서 예측값을 구한다.
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadDataSet(link + 'testSetRBF2.txt')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m): # 검사용 데이터 집합인 다른 데이터 집합을 사용해서 테스트
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print ("the test error rate is: %f" % (float(errorCount)/m))

코드의 검토를 위해 아래의 코드를 실행해보자.

In [0]:
testRbf()

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pairs changed 3
fullSet, iter: 0 i:3, pairs changed 4
fullSet, iter: 0 i:4, pairs changed 4
fullSet, iter: 0 i:5, pairs changed 5
fullSet, iter: 0 i:6, pairs changed 6
fullSet, iter: 0 i:7, pairs changed 6
fullSet, iter: 0 i:8, pairs changed 7
fullSet, iter: 0 i:9, pairs changed 7
fullSet, iter: 0 i:10, pairs changed 8
fullSet, iter: 0 i:11, pairs changed 9
fullSet, iter: 0 i:12, pairs changed 10
fullSet, iter: 0 i:13, pairs changed 11
fullSet, iter: 0 i:14, pairs changed 12
fullSet, iter: 0 i:15, pairs changed 13
fullSet, iter: 0 i:16, pairs changed 14
fullSet, iter: 0 i:17, pairs changed 14
fullSet, iter: 0 i:18, pairs changed 15
fullSet, iter: 0 i:19, pairs changed 16
fullSet, iter: 0 i:20, pairs changed 16
fullSet, iter: 0 i:21, pairs changed 17
fullSet, iter: 0 i:22, pairs changed 17
fullSet, iter: 0 i:23, pairs changed 17
fullSet, iter: 0 i:24, pairs changed 18
j not moving enough
fu

실행시켜보면, 매개변수 k1으로 검사 오류, 훈련 오류, 지지 벡터의 수가 k1에 따라 어떻게 변하는지 확인할 수 있다.

<br><br>
<h2><b>필기체 인식 예제 다시 적용하기</b></h2>

2장에서 kNN 알고리즘을 사용해서 필기체 인식 예제를 실행해보았다. 하지만 모든 훈련 예제를 메모리에 저장하고 있어야 하는 단점이 존재했다. SVM을 사용하면 kNN으로 할 때보다 **더 적은 예제를 저장하면서 KNN과 유사한 성능**을 낼 수 있다.

<br><br>
우리는 SVM을 통한 **이진 분류기**를 만들었기 때문에, 예제를 조금 변경하여 **1과 9를 구분하는** 이진 분류기를 만들 것이다.

<br>
우리가 만든 SVM에서는 1과 -1로 분류를 구분하기 때문에, <b>9는 -1, 1은 1로 구분</b>하도록 한다.

<br>
예제를 위해서 2장의 코드를 변경한 아래의 새 코드를 확인해보자.



In [0]:
def img2vector(filename): # 이미지를 벡터로 변환하는 코드
    returnVect = zeros((1,1024))
    fr = open(link + filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(link + dirName)           # 훈련 세트를 불러온다.
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels    

def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = loadImages('trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd];
    print ("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadImages('testDigits')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print ("the test error rate is: %f" % (float(errorCount)/m))

테스트를 위해 testDigits()를 실행시켜 보자. 커널은 rbf를 사용하고, gamma는 20으로 한다.

In [0]:
testDigits(('rbf', 20))

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pairs changed 3
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 3
fullSet, iter: 0 i:6, pairs changed 3
fullSet, iter: 0 i:7, pairs changed 4
fullSet, iter: 0 i:8, pairs changed 5
fullSet, iter: 0 i:9, pairs changed 5
fullSet, iter: 0 i:10, pairs changed 5
fullSet, iter: 0 i:11, pairs changed 6
fullSet, iter: 0 i:12, pairs changed 6
fullSet, iter: 0 i:13, pairs changed 7
fullSet, iter: 0 i:14, pairs changed 8
fullSet, iter: 0 i:15, pairs changed 9
fullSet, iter: 0 i:16, pairs changed 9
fullSet, iter: 0 i:17, pairs changed 10
fullSet, iter: 0 i:18, pairs changed 10
fullSet, iter: 0 i:19, pairs changed 11
j not moving enough
fullSet, iter: 0 i:20, pairs changed 11
fullSet, iter: 0 i:21, pairs changed 12
fullSet, iter: 0 i:22, pairs changed 13
fullSet, iter: 0 i:23, pairs changed 13
j not moving enough
fullSet, iter: 0 i:24, pair

<br><br><br><br>

<h2><b>요약</h2></b>
SVM은 분류기의 한 유형이며, 지지 벡터들은 학습하고자 하는 것에 대해 작은 일반화 오류를 가지도록 학습한다. 이러한 장점 때문에 SVM은 인기가 있으며 비지도 학습에서 가장 좋은 알고리즘 중 하나로 간주된다.